In [1]:
import sys
sys.path.append('../Texygen')
from utils.metrics.Bleu import Bleu
from utils.metrics.SelfBleu import SelfBleu

In [2]:
real_data_coco = 'data/testdata/processed_image_coco.txt'
real_data_emnlp = 'data/testdata/emnlp_news.txt'
def get_bleu_scores(test_data, is_self_bleu=False, grams=[2,3,4,5], dataset='coco'):
    result = []
    if dataset == 'coco':
        real_data = real_data_coco
    else:
        real_data = real_data_emnlp
    
    for gram in grams:
        if is_self_bleu:
            bleu = SelfBleu(test_text=test_data, gram=gram)
        else:    
            bleu = Bleu(test_text=test_data, real_text=real_data, gram=gram)
        result.append(bleu.get_score())
    return result

# present_rate = 0

In [26]:
fpath = 'save/coco_maskgan/coco_GAN_generated_200_real[00_00].txt'
print(get_bleu_scores(fpath))
print(get_bleu_scores(fpath, is_self_bleu=True))

[0.5623968569083401, 0.3182488210671706, 0.18138734736933074, 0.11191996403847103]
[0.7745205968594968, 0.5767489761689676, 0.39402653541058164, 0.2607846570229674]


In [24]:
fpath = 'save/coco_tokmangan/coco_GAN_generated_200_real[00_00].txt'
print(get_bleu_scores(fpath))
print(get_bleu_scores(fpath, is_self_bleu=True))

[0.607117323692313, 0.3616901304803233, 0.20705153281227345, 0.128581046529209]
[0.8373457499945866, 0.6540450503042257, 0.47061151463340506, 0.3198695469562022]


In [3]:
fpath = 'save/coco_tokmangan/coco_MLE_generated_80_real[00_00].txt'
print(get_bleu_scores(fpath))
print(get_bleu_scores(fpath, is_self_bleu=True))

[0.5193342850106948, 0.27458516988408527, 0.14814481723269093, 0.09199265669007906]
[0.7070811282518223, 0.4595938763837211, 0.2639753261327469, 0.15985081826815734]


In [25]:
fpath = 'data/testdata/processed_image_coco.txt'
print(get_bleu_scores(fpath))
print(get_bleu_scores(fpath, is_self_bleu=True))

[0.764394836320349, 0.5553374822148981, 0.3860167312371636, 0.2783596599333682]
[0.765516991569969, 0.5539745716689222, 0.3818106654017613, 0.25584138024659586]


# present_rate = 0.25

In [27]:
fpath = 'save/coco_maskgan/coco_GAN_generated_200_real[025_025].txt'
print(get_bleu_scores(fpath))
print(get_bleu_scores(fpath, is_self_bleu=True))

[0.5690013644832794, 0.312652007447638, 0.1717622090409478, 0.10584011794723397]
[0.707577789826631, 0.46842620591785206, 0.28816255079854936, 0.17936333752308142]


In [23]:
fpath = 'save/coco_tokmangan/coco_GAN_generated_200_real[025_025].txt'
print(get_bleu_scores(fpath))
print(get_bleu_scores(fpath, is_self_bleu=True))

[0.6050566189693724, 0.349949065715681, 0.19641372552847913, 0.1214566574856975]
[0.7735844313923919, 0.5525893155288543, 0.35559231542289405, 0.22117989686829417]


# present_rate = 0.5

In [28]:
fpath = 'save/coco_maskgan/coco_GAN_generated_200_real[05_05].txt'
print(get_bleu_scores(fpath))
print(get_bleu_scores(fpath, is_self_bleu=True))

[0.6234539892667136, 0.37356374322263197, 0.21759681257607497, 0.1367527616555995]
[0.6927043716940569, 0.4227775662794291, 0.24152653082065018, 0.14194592631443273]


In [22]:
fpath = 'save/coco_tokmangan/coco_GAN_generated_200_real[05_05].txt'
print(get_bleu_scores(fpath))
print(get_bleu_scores(fpath, is_self_bleu=True))

[0.6231231908847035, 0.3721841994177875, 0.21554327916962868, 0.1369216535246297]
[0.6965406342331433, 0.42204959577172074, 0.23357188292735231, 0.14072545454861354]


# present_rate = 0.75

In [29]:
fpath = 'save/coco_maskgan/coco_GAN_generated_200_real[075_075].txt'
print(get_bleu_scores(fpath))
print(get_bleu_scores(fpath, is_self_bleu=True))

[0.6827047737141388, 0.4500344934227466, 0.2851031707322801, 0.19057605428787733]
[0.7087943884167633, 0.4488333559516468, 0.26023982623067365, 0.15546056458582147]


In [21]:
fpath = 'save/coco_tokmangan/coco_GAN_generated_200_real[075_075].txt'
print(get_bleu_scores(fpath))
print(get_bleu_scores(fpath, is_self_bleu=True))

[0.6712028727191234, 0.43211479625793975, 0.269301303590116, 0.17969979212519768]
[0.7003819666858252, 0.43016476915900104, 0.24058175856478575, 0.14799400435799714]
